The purpose of this notebook is to demonstrate the use of the connect_pois tool to connect points (such as houses) to a road network, in order to be used for more precise accessibility calculations. 

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
#%% # get pois. which must not be multipoints, for connect_poi to work
folder = "C:\\Users\\z3258367\\OneDrive - UNSW\\#PhD\\"

poi_df = gpd.read_file(''.join(
    (folder + "Walkability\\Mavoa\\Pandana inputs\\Greater_Sydney_OSM_POIs_sp.shp")))
buildings = gpd.read_file(''.join((
    folder + "Data\\Colouring\\Building Footprints\\sydney_bf.shp")))

In [5]:
poi_df = poi_df.to_crs("EPSG:7856")
buildings_ctrs = buildings.to_crs("EPSG:7856")
buildings_ctrs.geometry = buildings_ctrs.geometry.centroid

In [7]:
#%% resolves multipoints if present and set indexes
properties_ex = buildings_ctrs.explode(column=None, ignore_index=True, index_parts=False)
properties_ex.reset_index(drop=True, inplace=True)
properties_ex['key'] = properties_ex.index

poi_ex = poi_df.explode(column=None, ignore_index=True, index_parts=False)
poi_ex.reset_index(drop=True, inplace=True)
poi_ex['poi_id'] = poi_ex.index

In [8]:
#%% # Import a clean network and convert it to a graph
GS = nx.read_shp("C:/Users/z3258367/OneDrive - UNSW/#PhD/Data/VA Cleaned OSM Greater Sydney Ped Aug2020/Greater_Sydney_Pedestrian_Cleaned.shp",simplify=False,geom_attrs=True) 

edges_df = nx.to_pandas_edgelist(GS)
edges_gdf = gpd.GeoDataFrame(edges_df, geometry = gpd.GeoSeries.from_wkt(edges_df['Wkt']))
edges_gdf = edges_gdf.set_crs("EPSG:3857",allow_override=True).to_crs("EPSG:7856")

nodes_df = pd.DataFrame(GS.nodes(data=True))
nodes_gdf = gpd.GeoDataFrame(nodes_df, geometry=gpd.points_from_xy(list(zip(*nodes_df[0]))[0],list(zip(*nodes_df[0]))[1]))
nodes_gdf = nodes_gdf.set_crs("EPSG:3857",allow_override=True).to_crs("EPSG:7856")

<ipython-input-8-90353826d5e3>:2: DeprecationWarning: read_shp is deprecated and will be removed in 3.0.See https://networkx.org/documentation/latest/auto_examples/index.html#geospatial.
  GS = nx.read_shp("C:/Users/z3258367/OneDrive - UNSW/#PhD/Data/VA Cleaned OSM Greater Sydney Ped Aug2020/Greater_Sydney_Pedestrian_Cleaned.shp",simplify=False,geom_attrs=True)


<class 'str'>


In [9]:
# for my new toolbox changes, need to establish a connect_id for the original nodes first.
nodes_gdf['connect_id'] = nodes_gdf.index

# Then use the tuple column to create from and to columns based on the 'osmid' in the edge list
node_Ids = pd.Series(nodes_gdf.connect_id.values, index=nodes_gdf[0]).to_dict()
edges_gdf['to'] = edges_gdf['target'].map(node_Ids)
edges_gdf['from'] = edges_gdf['source'].map(node_Ids)

# remove extraneous columns from nodes and edges
nodes_gdf = nodes_gdf.drop([0,1], axis = 1).copy()
edges_gdf = edges_gdf.drop(['source','target','Wkb','code','Json','Wkt'], axis = 1)

In [19]:
new_nodes.to_csv("new_nodes_test_colour_912.csv")
new_edges.to_csv("new_edge_test_colour_912.csv")
new_edges.to_file("C:/Users/z3258367/OneDrive - UNSW/#PhD/Walkability/Mavoa/Github/new_edge_test_colour_912.shp")

<ipython-input-19-6a7d7d58f48b>:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  new_edges.to_file("C:/Users/z3258367/OneDrive - UNSW/#PhD/Walkability/Mavoa/Github/new_edge_test_colour_912.shp")


In [20]:
import pandas as pd
import networkx as nx
import geopandas as gpd
from shapely import wkt

from toolbox_JR import connect_poi

In [21]:
#%% # run connect pois with property centroids
new_nodes, new_edges = connect_poi(properties_ex, nodes_gdf, edges_gdf, key_col='key', path=None, meter_epsg=7856)

#%% # run connect pois with POIs
new_nodes_2, new_edges_2 = connect_poi(poi_ex, new_nodes, new_edges, key_col='poi_id', path=None, meter_epsg=7855, prefix=8990000000)

Building rtree...
Updating external nodes...
Projecting POIs to the network...
Updating internal nodes...
Updating internal edges...
Missing 'to' nodes: 696
Updating external links...
Missing 'to' nodes: 0
Remove faulty projections: 15069/1318210 (1.14%)
NOTE: duplication in node coordinates keys
Nodes count: 3930298
Node coordinates key count: 3802963
Missing 'from' nodes: 679
Missing 'to' nodes: 696
Building rtree...
Updating external nodes...
Projecting POIs to the network...
Updating internal nodes...
Updating internal edges...
Missing 'to' nodes: 30
Updating external links...
Missing 'to' nodes: 0
Remove faulty projections: 53/24676 (0.21%)
NOTE: duplication in node coordinates keys
Nodes count: 3979650
Node coordinates key count: 3847520
Missing 'from' nodes: 699
Missing 'to' nodes: 723


In [22]:
new_edges_2[new_edges_2['to'].isna()]

,fclass,ref,Classifica,oneway,Shape_Leng,name,osm_id,bridge,tunnel,maxspeed,...,ShpName,Shape_Le_1,geometry,to,from,length,connect_id,key,connect_type,poi_id
1396672,residential,None,None,B,0.010638,Sugarloaf Crescent,2953495,F,F,0.0,...,Greater_Sydney_Pedestrian_Cleaned,1263.686253,"LINESTRING (334655.091 6258721.718, 334652.081...",NaN,3580281.0,3.442252,3580281.0_nan,NaN,NaN,NaN
1397030,residential,None,None,B,0.015884,Neerim Road,2953515,F,F,0.0,...,Greater_Sydney_Pedestrian_Cleaned,1833.073658,"LINESTRING (335043.042 6259993.479, 335050.216...",NaN,3836747.0,7.398081,3836747.0_nan,NaN,NaN,NaN
1399700,residential,None,None,B,0.001884,Payten Street,173301607,F,F,0.0,...,Greater_Sydney_Pedestrian_Cleaned,232.061898,"LINESTRING (326944.491 6238049.818, 326962.924...",NaN,3183905.0,28.021138,3183905.0_nan,NaN,NaN,NaN
1399815,residential,None,None,B,0.005896,Carlton Crescent,3680973,F,F,0.0,...,Greater_Sydney_Pedestrian_Cleaned,717.088156,"LINESTRING (326976.292 6238040.347, 326962.924...",NaN,3921235.0,17.720840,3921235.0_nan,NaN,NaN,NaN
1401390,residential,None,None,B,0.003639,Oberon Street,3738690,F,F,50.0,...,Greater_Sydney_Pedestrian_Cleaned,406.917437,"LINESTRING (337377.443 6244871.248, 337378.922...",NaN,2682890.0,1.495721,2682890.0_nan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4275947,service,None,None,B,0.003368,None,352728812,F,F,0.0,...,Greater_Sydney_Pedestrian_Cleaned,436.044823,"LINESTRING (359749.311 6310234.192, 359748.031...",NaN,3968962.0,6.892291,3968962.0_nan,NaN,NaN,NaN
4276133,service,None,None,B,0.000795,Park Lane,2654986,F,F,0.0,...,Greater_Sydney_Pedestrian_Cleaned,95.513658,"LINESTRING (332855.142 6249767.920, 332866.384...",NaN,3964825.0,15.136748,3964825.0_nan,NaN,NaN,NaN
4278723,service,None,None,B,0.001269,None,91986736,F,F,0.0,...,Greater_Sydney_Pedestrian_Cleaned,151.203520,"LINESTRING (318647.650 6275558.920, 318649.384...",NaN,3966936.0,2.151282,3966936.0_nan,NaN,NaN,NaN
4279671,cycleway,None,None,B,0.000602,None,395699901,F,F,0.0,...,Greater_Sydney_Pedestrian_Cleaned,69.207031,"LINESTRING (331758.118 6246993.623, 331751.533...",NaN,3957131.0,7.124344,3957131.0_nan,NaN,NaN,NaN


In [23]:
d_edges = new_edges_2[new_edges_2['to'].isin(new_nodes_2['connect_id']) & new_edges_2['from'].isin(new_nodes_2['connect_id'])]
len(d_edges)/len(new_edges_2)

0.9996385253665205

In [31]:
d_nodes = new_nodes_2.round(6).drop_duplicates(subset=['x', 'y'], ignore_index=True)

In [32]:
len(d_nodes)/len(new_nodes_2)

0.9668036133830864

In [33]:
#this no longer necessary since I fixed it in the toolbox

#%% # have to rematch the edges to the nodes, as some of them are the wrong precision
nodes_coord = d_nodes['geometry'].map(lambda x: wkt.loads(wkt.dumps(x, rounding_precision=6)).coords[0])
nodes_id_dict = dict(zip(nodes_coord, d_nodes['connect_id'].astype('int64')))
matched_edges = new_edges_2

# my version with new nodes
matched_edges['from'] = matched_edges['geometry'].map(lambda x: nodes_id_dict.get(list(wkt.loads(wkt.dumps(x, rounding_precision=6)).coords)[0], None))
matched_edges['to'] = matched_edges['geometry'].map(lambda x: nodes_id_dict.get(list(wkt.loads(wkt.dumps(x, rounding_precision=6)).coords)[-1], None))

matched_edges['from'] = matched_edges['from'].astype('Int64')
matched_edges['to'] = matched_edges['to'].astype('Int64')

dropped_edges_2 = matched_edges.dropna(subset=['from','to'])
print(len(matched_edges)/len(new_edges_2))
print(len(dropped_edges_2)/len(new_edges_2))

1.0
0.9983255742540582


This next cell is necessary otherwise there may eventually be errors when trying to run pandana. (See https://github.com/UDST/pandana/issues/88). All edges must reference existing nodes.

In [34]:
# drop NAs - this seems to be essential
dropped_edges = new_edges_2.dropna(subset=['from','to'])
print(len(dropped_edges)/len(new_edges_2))

# and this, this appears to have actually fixed things. though I'm not totally sure why.
dropped_edges = dropped_edges[dropped_edges['to'].isin(d_nodes['connect_id']) & dropped_edges['from'].isin(d_nodes['connect_id'])]
print(len(dropped_edges)/len(new_edges_2))

dropped_edges = dropped_edges.drop_duplicates(subset=['from','to'])

0.9983255742540582
0.9983255742540582
1.020713234950124


In [36]:
len(new_edges_2.dropna(subset=['from','to']))/len(new_edges_2)

0.9983255742540582

In [37]:
#%% export
new_nodes_2.to_csv("new_nodes_test_colour_1912.csv")
dropped_edges.to_csv("new_edge_test_colour_1912.csv")


<ipython-input-37-89c4100234ff>:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  dropped_edges.to_file("C:/Users/z3258367/OneDrive - UNSW/#PhD/Walkability/Mavoa/Github/new_edge_test_colour_1912.shp")


KeyboardInterrupt: 

In [38]:
d_nodes.to_csv("new_nodes_test_colour_1912_2.csv")

In [39]:
dropped_edges.to_file("C:/Users/z3258367/OneDrive - UNSW/#PhD/Walkability/Mavoa/Github/new_edge_test_colour_1912_2.shp")

<ipython-input-39-433510a325a9>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  dropped_edges.to_file("C:/Users/z3258367/OneDrive - UNSW/#PhD/Walkability/Mavoa/Github/new_edge_test_colour_1912_2.shp")
